In [1]:
from dgp import *
from estimation import *

# 1. Simulate or load data
adj = get_graph(2000, 1, 1, seed=2)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1.00,0.50,0.10,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y, A, L = sample_network(adj, tau, rho, nu, gamma, beta, num_iter=100000)
print(Y.shape, A.shape, L.shape, Y.mean(), A.mean(), L.mean())

(2000,) (2000,) (2000, 3) 0.3815 0.383 0.4535


In [2]:
from agcEffect import *

agc_effect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=30,
    burnin_R=10,
    seed=0
)

{'average': 0.32214999999999994,
 'direct_effect': -0.13916666666666666,
 'spillover_effect': -0.03300000000000003,
 'psi_gamma': 0.32214999999999994,
 'psi_1_gamma': 0.2801166666666666,
 'psi_0_gamma': 0.4192833333333333,
 'psi_zero': 0.4522833333333333}

In [11]:
from autognet2 import *

def evaluate_autognet_via_agc_effect_debug(adj_matrix, Y, A, L, treatment_allocation=0.5, R=10, burnin=5, seed=0):
    """
    Fit autognet models and evaluate causal effects using agc_effect.
    """
    models = fit_autog_models(Y, A, L, adj_matrix)
    tau, rho, nu, beta_hat = extract_parameters_from_autog_models(models, adj_matrix)
    
    # print("tau:", tau)
    # print("rho:", rho)
    # print("nu:", nu)
    # print("beta:", beta_hat)
    
    beta_true = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])
    diff =  beta_hat - beta_true
    print("beta_hat - beta for psi_zero:", np.mean(list(diff[:1])+list(diff[3:])))
    print("beta_hat - beta for psi_0_gamma:", np.mean(list(diff[:1])+list(diff[2:])))
    
    ret = agc_effect(
        adj_matrix=adj_matrix,
        tau=tau,
        rho=rho,
        nu=nu,
        beta=beta_hat,
        treatment_allocation=treatment_allocation,
        R=R,
        burnin_R=burnin,
        seed=seed
    )
    
    print("psi_zero:", ret['psi_zero'])

    return ret

evaluate_autognet_via_agc_effect_debug(adj, Y, A, L, treatment_allocation=0.7, R=30, burnin=10, seed=2)

beta_hat - beta for psi_zero: -0.027759634528917866
beta_hat - beta for psi_0_gamma: -0.0048975758027971095
psi_zero: 0.42321666666666663


{'average': 0.33695,
 'direct_effect': -0.12121666666666658,
 'spillover_effect': -0.0031333333333333213,
 'psi_gamma': 0.33695,
 'psi_1_gamma': 0.2988666666666667,
 'psi_0_gamma': 0.4200833333333333,
 'psi_zero': 0.42321666666666663}

In [12]:
from dgp import *
from estimation import *

# 1. Simulate or load data
adj = get_graph(2000, 1, 2, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1.00,0.50,0.10,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=20,
    burnin_R=10, seed=0)

100%|██████████| 30/30 [00:11<00:00,  2.68it/s]


In [ ]:
from autognet2 import *

ret = {
    'average': [],
    'direct_effect': [],
    'spillover_effect': [],
    'psi_0_gamma': [],
    'psi_zero': []
}
for Y, A, L in tqdm(zip(Y_chain, A_chain, L_chain)):
    ret_i = evaluate_autognet_via_agc_effect_debug(adj, Y, A, L, treatment_allocation=0.7, R=20, burnin=10, seed=1)
    ret['average'].append(ret_i['average'])
    ret['direct_effect'].append(ret_i['direct_effect'])
    ret['spillover_effect'].append(ret_i['spillover_effect'])
    ret['psi_0_gamma'].append(ret_i['psi_0_gamma'])
    ret['psi_zero'].append(ret_i['psi_zero'])
    
print('Average effect:', np.mean(ret['average']))
print('Direct effect:', np.mean(ret['direct_effect']))
print('Spillover effect:', np.mean(ret['spillover_effect']))
print('Psi_0_gamma:', np.mean(ret['psi_0_gamma']))
print('Psi_zero:', np.mean(ret['psi_zero']))

0it [00:00, ?it/s]

beta_hat - beta for psi_zero: 0.009367163662125852
beta_hat - beta for psi_0_gamma: 0.004246717130985263


1it [00:42, 42.96s/it]

psi_zero: 0.4842000000000001
beta_hat - beta for psi_zero: -0.014169341996112113
beta_hat - beta for psi_0_gamma: -6.339383969779233e-05


2it [01:22, 41.23s/it]

...

psi_zero: 0.46535000000000004
beta_hat - beta for psi_zero: -0.0041290371973626595
beta_hat - beta for psi_0_gamma: -0.001561259987297298


18it [12:57, 31.68s/it]

psi_zero: 0.46240000000000003
beta_hat - beta for psi_zero: 0.0037397840688457376
beta_hat - beta for psi_0_gamma: 0.0053376264582816685


19it [13:34, 33.20s/it]

psi_zero: 0.46335000000000004
beta_hat - beta for psi_zero: -0.012245982841408157
beta_hat - beta for psi_0_gamma: -0.012131894635677785
